# Input Testing
***
PVD Tools has a few parameters which can be directly taken from `pvlib` outputs. The following will be tested
1. poa_global
2. temp, dry_bulb
3. temp, dew_point
4. elevation
***

In [1]:
import pvlib
import numpy as np
import pandas as pd

import PVDegradationTools as PVD

From a psm3 file we need the following:
- DNI
- DHI
- GHI
- Cloud Cover
- Temperature, Dry-Bulb
- Temperature, Dew-Point
- Wind Speed
- Relative Humidity
- Latitude
- Longitude

In [2]:
sam_file = r'C:\Users\mbrown2\Downloads\psm3_test\145809_39.73_-105.18_1999.csv'

data, meta = pvlib.iotools.read_psm3(filename=sam_file)
data.columns

c:\users\mbrown2\documents\github\pvlib\pvlib\iotools\psm3.py:349: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(


Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'DHI', 'DNI', 'Dew Point',
       'Surface Albedo', 'Wind Speed', 'Relative Humidity', 'Temperature',
       'Pressure', 'GHI', 'Cloud Type', 'Fill Flag', 'Precipitable Water',
       'Wind Direction', 'Global Horizontal UV Irradiance (280-400nm)',
       'Global Horizontal UV Irradiance (295-385nm)', 'Clearsky DHI',
       'Clearsky DNI', 'Clearsky GHI', 'Solar Zenith Angle'],
      dtype='object')

In [3]:
cols = ['DNI','DHI','GHI','Temperature','Dew Point','Wind Speed','Relative Humidity']
df_view = data[cols]
dni = df_view.DNI
dhi = df_view.DHI
ghi = df_view.GHI

temp_dry = df_view.Temperature

time_range = df_view.index

solar_pos = pvlib.solarposition.get_solarposition(time=time_range, latitude=meta['Latitude'],
                                            longitude=meta['Longitude'], altitude=meta['Elevation'])

In [4]:
poa_df = pvlib.irradiance.get_total_irradiance(surface_tilt=20, surface_azimuth=270, 
                                                    solar_zenith=solar_pos.apparent_zenith,
                                                    solar_azimuth=solar_pos.azimuth,
                                                    dni=dni, ghi=ghi, dhi=dhi)
poa_global = poa_df.poa_global

#### Water Vapor Pressure

In [5]:
water_vapor_pressure = PVD.EnergyCalcs.water_vapor_pressure(df_view['Dew Point'])

avg_wvp = water_vapor_pressure.mean()
k = PVD.EnergyCalcs.k(avg_wvp=avg_wvp)

edge_seal_width = PVD.EnergyCalcs.edge_seal_width(k)

# Currently No Data: Cloud Cover
# dew_yield = PVD.EnergyCalcs.dew_yield(elevation=meta['Elevation'], dew_point=df_view['Dew Point']
#                                         dry_bulb=df_view['Temperature'],
#                                         wind_speed=df_view['Wind Speed'],
#                                         n=)

### VantHoff Degradation

In [6]:
I_chamber = 1000
temp_chamber = 60

cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

temp_cell = pvlib.temperature.sapm_cell(poa_global=poa_global, temp_air=df_view['Temperature'],
                                        wind_speed=df_view['Wind Speed'], **cell_params)

vantHoff_deg = PVD.EnergyCalcs.vantHoff_deg(I_chamber=I_chamber, poa_global=poa_global,
                                            temp_cell=temp_cell, temp_chamber=temp_chamber)

irr_weighted_avg_v = PVD.EnergyCalcs.IwaVantHoff(poa_global=poa_global, temp_cell=temp_cell)

### Arrhenius Degradation

In [7]:
rh_chamber = 15
Ea = 40

temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=df_view['Temperature'],
                                            wind_speed=df_view['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

rh_surface = PVD.RelativeHumidity.rh_surface_outside(rh_ambient=df_view['Relative Humidity'],
                                                        temp_ambient=df_view['Temperature'],
                                                        temp_surface=temp_module)

arrhenius_deg = PVD.EnergyCalcs.arrhenius_deg(I_chamber=I_chamber, rh_chamber=rh_chamber,
                                              rh_outdoor=rh_surface, poa_global=poa_global,
                                              temp_chamber=temp_chamber, temp_cell=temp_cell,
                                              Ea=Ea)

irr_weighted_avg_a = PVD.EnergyCalcs.IwaArrhenius(poa_global=poa_global, 
                                                  rh_outdoor=df_view['Relative Humidity'],
                                                  temp_cell=temp_cell, Ea=Ea)

### Relative Humidity

In [8]:
rh_front_encap = PVD.RelativeHumidity.rh_front_encap(rh_ambient=df_view['Relative Humidity'],
                                                    temp_ambient=df_view['Temperature'],
                                                    temp_surface=temp_module)

rh_back_encap = PVD.RelativeHumidity.rh_back_encap(rh_ambient=df_view['Relative Humidity'],
                                                    temp_ambient=df_view['Temperature'],
                                                    temp_surface=temp_module)

rh_backsheet = PVD.RelativeHumidity.rh_backsheet(rh_back_encap=rh_back_encap,
                                                    rh_surface_outside=rh_surface)